<a href="https://colab.research.google.com/github/mathissen/streamlit-ea-app/blob/master/202103_Venue_Data_Package_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Venue Data Package Metrics

This notebook contains definition of the Venue package metrics into the Portal.

The following metrics are present:

- State Retail Impact by State
- Venue Capture Rate
- Venue Average Catchment Area Distance 
- Venue Catchment Area Distance Median
- Venue Foot Traffic [Day / Week / Month]
- Venue Visit Length Median
- Venue Traffic Patterns Day of Week
- Venue Traffic Patterns Day of Week Hour


---

## Setup

In [ ]:
# Authenticate
PROJECT_ID = "uc-unacat" #@param {type: "string"}


COST_LIMIT_USD =   10#@param {type: "number"}

LOCATION = "US" #@param {type: "string"}

from google.cloud import bigquery
from google.colab import auth

from datetime import datetime

auth.authenticate_user()
client = bigquery.Client(PROJECT_ID)

# Cost estimates

def MB(bytes):
  return float(bytes) / 2**20


def GB(bytes):
  return float(bytes) / 2**30


def TB(bytes):
  return float(bytes) / 2**40


def USD(bytes):
  return (5.00 * TB(max(bytes, 10 * (2 ** 20))))


def estimate_query_bytes(query):
  
  job_config = bigquery.QueryJobConfig()
  job_config.dry_run = True
  query_job = client.query(
      query,
      location=LOCATION,
      job_config=job_config
  )  # API request
  
  return query_job.total_bytes_processed



# Custom bigquery magics that doesn't make dataframe
from __future__ import print_function

import time
from concurrent import futures

try:
    import IPython
    from IPython import display
    from IPython.core import magic_arguments
except ImportError:  # pragma: NO COVER
    raise ImportError('This module can only be loaded in IPython.')

import google.auth
from google.cloud import bigquery

class Context(object):
    """Storage for objects to be used throughout an IPython notebook session.
    A Context object is initialized when the ``magics`` module is imported,
    and can be found at ``google.cloud.bigquery.magics.context``.
    """
    def __init__(self):
        self._credentials = None
        self._project = None

    @property
    def credentials(self):
        """google.auth.credentials.Credentials: Credentials to use for queries
        performed through IPython magics
        Note:
            These credentials do not need to be explicitly defined if you are
            using Application Default Credentials. If you are not using
            Application Default Credentials, manually construct a
            :class:`google.auth.credentials.Credentials` object and set it as
            the context credentials as demonstrated in the example below. See
            `auth docs`_ for more information on obtaining credentials.
        Example:
            Manually setting the context credentials:
            >>> from google.cloud.bigquery import magics
            >>> from google.oauth2 import service_account
            >>> credentials = (service_account
            ...     .Credentials.from_service_account_file(
            ...         '/path/to/key.json'))
            >>> magics.context.credentials = credentials
        .. _auth docs: http://google-auth.readthedocs.io
            /en/latest/user-guide.html#obtaining-credentials
        """
        if self._credentials is None:
            self._credentials, _ = google.auth.default()
        return self._credentials

    @credentials.setter
    def credentials(self, value):
        self._credentials = value

    @property
    def project(self):
        """str: Default project to use for queries performed through IPython
        magics
        Note:
            The project does not need to be explicitly defined if you have an
            environment default project set. If you do not have a default
            project set in your environment, manually assign the project as
            demonstrated in the example below.
        Example:
            Manually setting the context project:
            >>> from google.cloud.bigquery import magics
            >>> magics.context.project = 'my-project'
        """
        if self._project is None:
            _, self._project = google.auth.default()
        return self._project

    @project.setter
    def project(self, value):
        self._project = value


context = Context()


def _run_query(client, query, job_config=None):
    """Runs a query while printing status updates
    Args:
        client (google.cloud.bigquery.client.Client):
            Client to bundle configuration needed for API requests.
        query (str):
            SQL query to be executed. Defaults to the standard SQL dialect.
            Use the ``job_config`` parameter to change dialects.
        job_config (google.cloud.bigquery.job.QueryJobConfig, optional):
            Extra configuration options for the job.
    Returns:
        google.cloud.bigquery.job.QueryJob: the query job created
    Example:
        >>> client = bigquery.Client()
        >>> _run_query(client, "SELECT 17")
        Executing query with job ID: bf633912-af2c-4780-b568-5d868058632b
        Query executing: 1.66s
        Query complete after 2.07s
        'bf633912-af2c-4780-b568-5d868058632b'
    """
    start_time = time.time()
    query_job = client.query(query, job_config=job_config)
    print('Executing query with job ID: {}'.format(query_job.job_id))

    while True:
        print('\rQuery executing: {:0.2f}s'.format(
            time.time() - start_time), end='')
        try:
            query_job.result(timeout=0.5)
            break
        except futures.TimeoutError:
            continue
    print('\nQuery complete after {:0.2f}s'.format(time.time() - start_time))
    return query_job


@magic_arguments.magic_arguments()
@magic_arguments.argument(
    'destination_var',
    nargs='?',
    help=('If provided, save the output to this variable in addition '
          'to displaying it.'))
@magic_arguments.argument(
    '--project',
    type=str,
    default=PROJECT_ID,
    help=('Project to use for executing this query. Defaults to the context '
          'project.'))
@magic_arguments.argument(
    '--use_legacy_sql',
    action='store_true',
    default=False,
    help=('Sets query to use Legacy SQL instead of Standard SQL. Defaults to '
          'Standard SQL if this argument is not used.'))
@magic_arguments.argument(
    '--no_df',
    action='store_true',
    default=False,
    help=('Sets query to not return result as dataframe'))
@magic_arguments.argument(
    '--cost_approved',
    action='store_true',
    default=False,
    help=('If set, do not prompt user to confirm execution of expensive queries.'))
@magic_arguments.argument(
    '--verbose', 
    action='store_true',
    default=False,
    help=('If set, print verbose output, including the query job ID and the '
          'amount of time for the query to finish. By default, this '
          'information will be displayed as the query runs, but will be '
          'cleared after the query is finished.'))
@magic_arguments.argument(
    '--param',
    type=str,
    default=None,
    help=('String value of optional query parameter @param'))
def _cell_magic(line, query):
    """Underlying function for bigquery cell magic
    Note:
        This function contains the underlying logic for the 'bigquery' cell
        magic. This function is not meant to be called directly.
    Args:
        line (str): "%%bigquery" followed by arguments as required
        query (str): SQL query to run
    Returns:
        pandas.DataFrame: the query results.
    """
    args = magic_arguments.parse_argstring(_cell_magic, line)

    project = args.project or context.project
    client = bigquery.Client(project=project, credentials=context.credentials)
    job_config = bigquery.job.QueryJobConfig()

    query_params = [
        bigquery.ScalarQueryParameter('param', 'STRING', args.param),
    ]
    
    job_config.query_parameters = query_params
    job_config.use_legacy_sql = args.use_legacy_sql
    
    # Check query cost first
    if not args.cost_approved:
      estimate = estimate_query_bytes(query)
      
      if USD(estimate) >= COST_LIMIT_USD:
        confirm = raw_input("Query costs up to ${:.2f}, ok? (y): ".format(USD(estimate)))
        if confirm != "y":
          print("Query aborted")
          return None
      
    query_job = _run_query(client, query, job_config)

    if not args.verbose:
        display.clear_output()

    query_job
    
    if args.no_df:
      print("Query successful. Cost ${:.2f}".format(USD(query_job.total_bytes_processed)))
      return None
    else:
      result = query_job.to_dataframe()
      print("Query successful. Cost ${:.2f}".format(USD(query_job.total_bytes_processed)))
      if args.destination_var:
          IPython.get_ipython().push({args.destination_var: result})
      return result
    
  
ip = get_ipython()

ip.register_magic_function(_cell_magic, magic_kind='cell', magic_name='bigquery')


# Print success message

print("Ran at", datetime.now(), "GMT")

Ran at 2021-04-29 10:19:27.133161 GMT


### Install

In [ ]:
%%capture

project_id = 'uc-unacat'

from google.colab import auth
auth.authenticate_user()


!gcloud config set project {project_id}

!gsutil cp gs://unacatlib/unacatlib-0.0.9-py2.py3-none-any.whl .

In [ ]:
%%capture

import sys
!{sys.executable} -m pip install --upgrade --force-reinstall unacatlib-0.0.9-py2.py3-none-any.whl;

In [ ]:
token_res = !gcloud auth print-identity-token
token = token_res[0]

In [ ]:
import datetime
import pprint

from unacatlib.client import Client
from unacatlib.address_component_builder import AddressComponentBuilder

from unacatlib.layer_builder import LayerBuilder
from unacatlib.metric_builder import MetricBuilder

from unacatlib.unacast.maps.v1 import ComponentKind
from unacatlib.unacast.metric.v1 import Cadence, ValueKind, ValueSpec, DimensionSpec
from unacatlib.index_job import IndexJob



#### Load catalog

In [ ]:
rwg_client = Client(server_address='dataops-api-server-snr3asztcq-uk.a.run.app', token=token)
rwg = rwg_client.catalog("rwg")


## Construct a BigQuery client object.
#client = bigquery.Client(PROJECT_ID)

#### Overview

In [ ]:
for ix, metric in enumerate(rwg.list_metrics()):
  print(str(ix).ljust(4), metric.id)

0    rwg_cbg_foot_traffic_day_202009
1    rwg_cbg_foot_traffic_month_202009
2    rwg_cbg_foot_traffic_week_202009
3    rwg_cbg_return_rate_month_202009
4    tract_migration_flow_month_202012
5    county_migration_flow_month_202012
6    rwg_county_social_distancing_grade_day
7    rwg_state_retail_foot_traffic_day_202009
8    rwg_state_social_distancing_grade_day
9    rwg_venue_capture_rate_quarter_202009
10   rwg_venue_foot_traffic_day_202009
11   rwg_venue_foot_traffic_month_202009
12   rwg_venue_foot_traffic_week_202009
13   rwg_venue_return_rate_month_202009
14   rwg_venue_visit_length_quarter_202009


## Metric-specific Dimensions

## Metrics

### Venue Return Rate Month

Fraction of people visiting a venue in the previous month which are also seen at the same venue this month.

In [ ]:
venue_layer = rwg.layer("unacast_poi_202102")

In [ ]:
venue_layer.id

'unacast_poi_202102'

In [ ]:
# main values
return_rate = ValueSpec(name="return_rate", 
                        unit="Fraction", 
                        value_kind=ValueKind.NUMBER, 
                        display_name="Return Rate", 
                        description="The share of observed devices seen in the previous month which are also seen this month."
                        )


In [ ]:
venue_return_rate_month = rwg.build_metric("rwg_venue_return_rate_month_202103") \
  .with_layer(venue_layer) \
  .with_value(return_rate) \
  .with_cadence(Cadence.MONTHLY) \
  .with_display_name("Venue Return Rate Month") \
  .with_description("Measure of the rate at which people are returning visitors to a venue.") \
  .create() 

In [ ]:
%%bigquery

  DECLARE START_DATE DATE DEFAULT "2018-10-01";

  CREATE OR REPLACE TABLE `uc-unacat.inbox.rwg_venue_return_rate_month_202103` AS (
    SELECT 
      venue_id as feature_id,
      observation_start_date as observation_start,
      observation_end_date as observation_end,
      COALESCE( frac , 0) as value_return_rate
    FROM `uc-catalog.venue_package_202103_4d_lag.venue_return_rate_month`
    WHERE observation_start_date >= START_DATE
         # AND venue_id != "84ba1736-181c-4be4-86cb-a1a514b4371e"
) 

Query successful. Cost $0.01


""


In [ ]:
%%bigquery

  select min(observation_start) as min_start, max(observation_end) as max_end from  `uc-unacat.inbox.rwg_venue_return_rate_month_202103`

Query successful. Cost $0.00


,min_start,max_end
0,2018-10-01,2021-03-31


In [ ]:
import datetime
index_job = venue_return_rate_month.index(
    big_query_table_id="rwg_venue_return_rate_month_202103",
    start_date=datetime.date(2018,10,1),
    end_date=datetime.date(2021,3,31),
    change_set=False
)

In [ ]:
index_job.status()

GetJobStatusResponse(job_id='eyJJbmRleElEIjoiYzI0NWR1cTIzYWtnMDBlZzg1cTAiLCJTdGFydERhdGUiOnsieWVhciI6MjAxOCwibW9udGgiOjEwLCJkYXkiOjF9LCJFbmREYXRlIjp7InllYXIiOjIwMjEsIm1vbnRoIjozLCJkYXkiOjMxfX0=', tasks=1)

In [ ]:
rwg.metric("rwg_venue_return_rate_month_202103").report()

MetricReport(observation_period=Period(start=Date(year=2018, month=10, day=1), end=Date(year=2021, month=3, day=31)), total_size=6873287)

---

### Capture Rate

In [ ]:
venue_layer = rwg.layer("unacast_poi_202102")

In [ ]:
# main values
capture_rate_p50 = ValueSpec(name="capture_rate_p50", 
                             unit="Fraction", 
                             value_kind=ValueKind.NUMBER, 
                             display_name="Median ratio between visits of venue and visits of vicinity.", 
                             description="50th percentile ratio between visits to the venue and visits to the area within 150m from the center of the venue."
                            )

In [ ]:
# supporting values
capture_rate_p25 = ValueSpec(name="capture_rate_p25", 
                             unit="Fraction", 
                             value_kind=ValueKind.NUMBER, 
                             display_name="25th percentile of ratio between visits of venue and visits of vicinity.", 
                             description="25th percentile ratio between visits to the venue and visits to the area within 150m from the center of the venue."
                            )

capture_rate_p75 = ValueSpec(name="capture_rate_p75", 
                             unit="Fraction", 
                             value_kind=ValueKind.NUMBER, 
                             display_name="75th percentile of ratio between visits of venue and visits of vicinity.", 
                             description="75th percentile ratio between visits to the venue and visits to the area within 150m from the center of the venue."
                            )


In [ ]:
venue_capture_rate = rwg.build_metric("rwg_venue_capture_rate_quarter_202103") \
                               .with_layer(venue_layer) \
                               .with_value(capture_rate_p50) \
                               .with_supporting_value(capture_rate_p25) \
                               .with_supporting_value(capture_rate_p75) \
                               .with_cadence(Cadence.QUARTERLY) \
                               .with_display_name("Venue Capture Rate") \
                               .with_description("The ratio between visits to the venue and visits to the area within 150m from the center of the venue.") \
                               .create() 

In [ ]:
%%bigquery

  DECLARE START_DATE DATE DEFAULT "2018-10-01";

  CREATE OR REPLACE TABLE `uc-unacat.inbox.rwg_venue_capture_rate_quarter_202103` AS (
    SELECT
      venue_id as feature_id,
      observation_start_date as observation_start,
      observation_end_date as observation_end,
      COALESCE(capture_rate_p50, 0) as value_capture_rate_p50,
      COALESCE(capture_rate_p25, 0) as value_capture_rate_p25,
      COALESCE(capture_rate_p75, 0) as value_capture_rate_p75
    FROM `uc-catalog.venue_package_202103_4d_lag.venue_capture_rate`
    WHERE observation_start_date >= START_DATE
      AND venue_id NOT IN ("84ba1736-181c-4be4-86cb-a1a514b4371e", "f8f3387d-72fc-490f-ac67-10fe7a2a2b2f", "70a6a8ed-855a-40ee-81d5-cd0d149e9bc5", "6075d9cc-9bed-448c-8895-87b9535c503c")
  )
  

Query successful. Cost $0.01


""


In [ ]:
%%bigquery

  select min(observation_start) as min_start, max(observation_end) as max_end from  `uc-unacat.inbox.rwg_venue_capture_rate_quarter_202103`

Query successful. Cost $0.00


,min_start,max_end
0,2018-10-01,2021-03-31


In [ ]:
import datetime
venue_capture_rate = rwg.metric("rwg_venue_capture_rate_quarter_202103") 

index_job = venue_capture_rate.index(
    big_query_table_id="rwg_venue_capture_rate_quarter_202103",
    start_date=datetime.date(2018,10,1),
    end_date=datetime.date(2021,3,31),
    change_set=False
)

In [ ]:
index_job.status()

GetJobStatusResponse(job_id='eyJJbmRleElEIjoiYzI0NWV0cTIzYWtnMDBlZzg1cWciLCJTdGFydERhdGUiOnsieWVhciI6MjAxOCwibW9udGgiOjEwLCJkYXkiOjF9LCJFbmREYXRlIjp7InllYXIiOjIwMjEsIm1vbnRoIjozLCJkYXkiOjMxfX0=', tasks=1)

In [ ]:
rwg.metric("rwg_venue_capture_rate_quarter_202103").report()

MetricReport(observation_period=Period(start=Date(year=2018, month=10, day=1), end=Date(year=2021, month=3, day=31)), total_size=4887956)

---

### Traffic Trends

* Traffic trends describes a time-series of visitation counts, across various aggregation periods (e.g., weekly, monthly).

* We use aggregated daily unique numbers of visitors by venue from the daily table to derive monthly, weekly  summed visitors and median daily visitors.

#### Venue Foot Traffic Day

In [ ]:
venue_layer = rwg.layer("unacast_poi_202102")

In [ ]:
# main values
total_people = ValueSpec(name="person_count", 
                         unit="People", 
                         value_kind=ValueKind.COUNT, 
                         display_name="People Count", 
                         description="The total number of people measured to visit the venue.")

In [ ]:
venue_foot_traffic_day = rwg.build_metric("rwg_venue_foot_traffic_day_202103") \
                            .with_layer(venue_layer) \
                            .with_value(total_people) \
                            .with_cadence(Cadence.DAILY) \
                            .with_display_name("Venue Foot Traffic Day") \
                            .with_description("The estimated number of people having visited a venue on a daily basis") \
                            .create()

In [ ]:
%%bigquery

  DECLARE START_DATE DATE DEFAULT "2018-11-01";

  CREATE OR REPLACE TABLE `uc-unacat.inbox.rwg_venue_foot_traffic_day_202103` AS (
    SELECT 
      venue_id as feature_id,
      local_event_date as observation_start,
      local_event_date as observation_end,
      person_count as value_person_count
    FROM `uc-catalog.venue_package_202103_4d_lag.venue_traffic_trends_day`  
    WHERE local_event_date >= START_DATE
          AND venue_id NOT IN ("84ba1736-181c-4be4-86cb-a1a514b4371e", "f8f3387d-72fc-490f-ac67-10fe7a2a2b2f", "70a6a8ed-855a-40ee-81d5-cd0d149e9bc5", "6075d9cc-9bed-448c-8895-87b9535c503c")
  )

In [ ]:
%%bigquery

  select min(observation_start) as min_start, max(observation_end) as max_end from  `uc-unacat.inbox.rwg_venue_foot_traffic_day_202103`

Query successful. Cost $0.05


,min_start,max_end
0,2018-11-01,2021-04-21


In [ ]:
import datetime
venue_foot_traffic_day = rwg.metric("rwg_venue_foot_traffic_day_202103")

index_job = venue_foot_traffic_day.index(
    big_query_table_id="rwg_venue_foot_traffic_day_202103",
    start_date=datetime.date(2018,11,1),
    end_date=datetime.date(2021,4,21),
    change_set=False
)

In [ ]:
index_job.wait()

⠋ 0/1 Tasks Completed

KeyboardInterrupt: ignored

In [ ]:
rwg.metric("rwg_venue_foot_traffic_day_202103").report()

MetricReport(observation_period=Period(start=Date(year=2018, month=11, day=1), end=Date(year=2019, month=5, day=25)), total_size=112554294)

#### Venue Foot Traffic Week

In [ ]:
venue_layer = rwg.layer("unacast_poi_202102")

In [ ]:
people_breakdown = rwg.dimension("people_breakdown")

In [ ]:
# main values
total_people = ValueSpec(name="person_count", 
                         unit="People", 
                         value_kind=ValueKind.COUNT, 
                         display_name="Total People Count", 
                         description="The total number of people measured to visit the venue.")

In [ ]:
# supporting values
daily_people_p50 = ValueSpec(name="person_count_p50", 
                         unit="People", 
                         value_kind=ValueKind.COUNT, 
                         display_name="Daily Median People Count", 
                         description="The median number of people measured to visit the venue per day in week.")

In [ ]:
venue_foot_traffic_week = rwg.build_metric("rwg_venue_foot_traffic_week_202103") \
                             .with_layer(venue_layer) \
                             .with_value(total_people) \
                             .with_supporting_value(daily_people_p50) \
                             .with_cadence(Cadence.WEEKLY) \
                             .with_display_name("Venue Foot Traffic Week") \
                             .with_description("The estimated number of people having visited a venue on a weekly basis") \
                             .create()

In [ ]:
%%bigquery

  # uc-prox-core-dev.portal_views.cbg_traffic_trends_week

  DECLARE START_DATE DATE DEFAULT "2018-10-01";

  CREATE OR REPLACE TABLE `uc-unacat.inbox.rwg_venue_foot_traffic_week_202103` AS (
    SELECT 
      venue_id as feature_id,
      observation_start_date as observation_start,
      observation_end_date as observation_end,
      person_count_sum as value_person_count,
      daily_person_count_p50	as value_person_count_p50,
    FROM `uc-catalog.venue_package_202103_4d_lag.venue_traffic_trends_week`
    WHERE observation_start_date >= START_DATE
      AND venue_id NOT IN ("84ba1736-181c-4be4-86cb-a1a514b4371e", "f8f3387d-72fc-490f-ac67-10fe7a2a2b2f", "70a6a8ed-855a-40ee-81d5-cd0d149e9bc5", "6075d9cc-9bed-448c-8895-87b9535c503c")
)

Query successful. Cost $0.23


""


In [ ]:
%%bigquery

  select min(observation_start) as min_start, max(observation_end) as max_end from  `uc-unacat.inbox.rwg_venue_foot_traffic_week_202103`

Query successful. Cost $0.01


,min_start,max_end
0,2018-11-04,2021-04-17


In [ ]:
import datetime
index_job = venue_foot_traffic_week.index(
    big_query_table_id="rwg_venue_foot_traffic_week_202103",
    start_date=datetime.date(2018, 11, 4),
    end_date=datetime.date(2021, 4, 17),
    change_set=False
)

In [ ]:
rwg.metric("rwg_venue_foot_traffic_week_202103").report()

MetricReport(observation_period=Period())

#### Venue Foot Traffic Month

In [ ]:
venue_layer = rwg.layer("unacast_poi_202102")

In [ ]:
# main values
total_people = ValueSpec(name="person_count", 
                         unit="People", 
                         value_kind=ValueKind.COUNT, 
                         display_name="Total People Count", 
                         description="The total number of people measured to visit the venue.")

In [ ]:
# supporting values
daily_people_p50 = ValueSpec(name="person_count_p50", 
                         unit="People", 
                         value_kind=ValueKind.COUNT, 
                         display_name="Daily Median People Count", 
                         description="The median number of people measured to visit the venue per day in a month.")

In [ ]:
venue_traffic_month = rwg.build_metric("rwg_venue_foot_traffic_month_202103") \
                         .with_layer(venue_layer) \
                         .with_value(total_people) \
                         .with_supporting_value(daily_people_p50) \
                         .with_cadence(Cadence.MONTHLY) \
                         .with_display_name("Venue Foot Traffic Month") \
                         .with_description("The estimated number of people having visited a venue on a monthly basis") \
                         .create()

In [ ]:
%%bigquery

  # uc-prox-core-dev.portal_views.cbg_foot_traffic_month

  DECLARE START_DATE DATE DEFAULT "2018-10-01";

  CREATE OR REPLACE TABLE `uc-unacat.inbox.rwg_venue_foot_traffic_month_202103` AS (
    SELECT 
      venue_id as feature_id,
      observation_start_date as observation_start,
      observation_end_date as observation_end,
      person_count_sum as value_person_count,
      daily_person_count_p50	as value_person_count_p50,
    FROM `uc-catalog.venue_package_202103_4d_lag.venue_traffic_trends_month`  
    WHERE observation_start_date >= "2018-10-01"
      AND venue_id NOT IN ("84ba1736-181c-4be4-86cb-a1a514b4371e", "f8f3387d-72fc-490f-ac67-10fe7a2a2b2f", "70a6a8ed-855a-40ee-81d5-cd0d149e9bc5", "6075d9cc-9bed-448c-8895-87b9535c503c")
)

Query successful. Cost $0.23


""


In [ ]:
%%bigquery

  select min(observation_start) as min_start, max(observation_end) as max_end from  `uc-unacat.inbox.rwg_venue_foot_traffic_month_202103`

Query successful. Cost $0.00


,min_start,max_end
0,2018-10-01,2021-03-31


In [ ]:
index_job = venue_traffic_month.index(
    big_query_table_id="rwg_venue_foot_traffic_month_202103",
    start_date=datetime.date(2018,10,1),
    end_date=datetime.date(2021,3,31),
    change_set=False
)

In [ ]:
rwg.metric("rwg_venue_foot_traffic_month_202103").report()

⠏ 0/1 Tasks Completed

MetricReport(observation_period=Period(start=Date(year=2018, month=10, day=1), end=Date(year=2021, month=3, day=31)), total_size=26983418)

⠋ 0/1 Tasks Completed

---

### Venue Average Catchment Area Distance


Distribution of distances traveled by people from their home and work areas to a venue.

In [ ]:
venue_layer = rwg.layer("unacast_poi_202102")

In [ ]:
habit_area_type = rwg.dimension('habit_area_type')

In [ ]:
  habit_area_type = rwg.build_dimension("habit_area_type") \
  .with_display_name("Types of habit area") \
  .create()

In [ ]:
distance_bin = rwg.dimension('distance_bin')

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE `uc-unacat.inbox.rwg_dimension_distance_bin` AS ( 

  SELECT
  DISTINCT distance_bin AS value,
  REPLACE(REPLACE(REPLACE(distance_bin,"50_ABOVE_KM","Above 50 km"),"_KM"," km"),"_"," - ") AS display_name
FROM
  `uc-catalog.venue_package_202103_4d_lag.venue_catchment_area_distance`
WHERE
  observation_start_date = DATE(2020,1,1)

)

Query successful. Cost $0.00


""


In [ ]:
distance_bin.index("rwg_dimension_distance_bin")

In [ ]:
# main values
median_distance = ValueSpec(name="distance_median", 
                            unit="Km", 
                            value_kind=ValueKind.NUMBER, 
                            display_name="Median Distance", 
                            description="Median distance of visitors' habit area (HOME/WORK) to selected venue.")

In [ ]:
# supporting values
distance_p25 = ValueSpec(name="distance_p25", 
                         unit="Km", 
                         value_kind=ValueKind.NUMBER, 
                         display_name="25th Percentile of Distance", 
                         description="25th percentile of distance of visitors' habit area (HOME/WORK) to selected venue.")

distance_p75 = ValueSpec(name="distance_p75", 
                         unit="Km", 
                         value_kind=ValueKind.NUMBER, 
                         display_name="75th Percentile of Distance", 
                         description="75th percentile of distance of visitors' habit area (HOME/WORK) to selected venue.")

fraction = ValueSpec(name="people_fraction", 
                          unit="Fraction", 
                          value_kind=ValueKind.NUMBER, 
                          display_name="Fraction of people in range distance range", 
                          description="Fraction of people whose habit area (home/work) is in the distance range")

In [ ]:
venue_catchment_area_distance = rwg.build_metric("rwg_venue_catchment_area_distance_quarter_202103") \
                               .with_layer(venue_layer) \
                               .with_dimension(habit_area_type, default_value="Home") \
                               .with_value(median_distance) \
                               .with_supporting_value(distance_p25) \
                               .with_supporting_value(distance_p75) \
                               .with_dimension(distance_bin, default_value="0_5_KM") \
                               .with_supporting_value(fraction) \
                               .with_cadence(Cadence.QUARTERLY) \
                               .with_display_name("Venue Average Catchment Area Distance") \
                               .with_description("Median distance to home/work of visitors in selected venue.") \
                               .create() 

In [ ]:
venue_catchment_area_distance = rwg.metric("rwg_venue_catchment_area_distance_quarter_202103")

In [ ]:
%%bigquery

  DECLARE START_DATE DATE DEFAULT "2018-10-01";

  CREATE OR REPLACE TABLE `uc-unacat.inbox.rwg_venue_catchment_area_distance_quarter_202103` AS (

      SELECT 
          venue_id as feature_id,
          habit_area_type as dimension_habit_area_type,
          distance_bin as dimension_distance_bin, 
          observation_start_date as observation_start,
          observation_end_date as observation_end,
          COALESCE(distance_from_habit_area_p50, 0) as value_distance_median, 
          COALESCE(distance_from_habit_area_p25, 0) as value_distance_p25, 
          COALESCE(distance_from_habit_area_p75, 0) as value_distance_p75, 
          fraction as value_people_fraction
     

      FROM `uc-catalog.venue_package_202103_4d_lag.venue_catchment_area_distance`
      WHERE observation_start_date >= START_DATE
        AND venue_id NOT IN ("84ba1736-181c-4be4-86cb-a1a514b4371e", "f8f3387d-72fc-490f-ac67-10fe7a2a2b2f", "70a6a8ed-855a-40ee-81d5-cd0d149e9bc5", "6075d9cc-9bed-448c-8895-87b9535c503c")
) 

Query successful. Cost $0.02


""


In [ ]:
%%bigquery

  select min(observation_start) as min_start, max(observation_end) as max_end from  `uc-unacat.inbox.rwg_venue_catchment_area_distance_quarter_202103`

Query successful. Cost $0.01


,min_start,max_end
0,2018-10-01,2021-03-31


In [ ]:
import datetime
index_job = venue_catchment_area_distance.index(
    big_query_table_id="rwg_venue_catchment_area_distance_quarter_202103",
    start_date=datetime.date(2018,10,1),
    end_date=datetime.date(2021,3,31),
    change_set=False
)

In [ ]:
index_job.status()

GetJobStatusResponse(job_id='eyJJbmRleElEIjoiYzI1NzdwYTIzYWtnMDBlcjR0ajAiLCJTdGFydERhdGUiOnsieWVhciI6MjAxOCwibW9udGgiOjEwLCJkYXkiOjF9LCJFbmREYXRlIjp7InllYXIiOjIwMjEsIm1vbnRoIjozLCJkYXkiOjMxfX0=', tasks=1)

In [ ]:
rwg.metric("rwg_venue_catchment_area_distance_quarter_202103").report()

GRPCError: ignored

---

### Venue Visit Length

Distribution of visit length for a specific venue in minutes.

In [ ]:
venue_layer = rwg.layer("unacast_poi_202102")

In [ ]:
# main values
visit_length_p50 = ValueSpec(name="visit_length_p50", 
                             unit="Minutes", 
                             value_kind=ValueKind.NUMBER, 
                             display_name="Median Visit Length (Min)", 
                             description="Estimated average length (median) of time spent visiting the venue per person in minutes."
                            )

In [ ]:
# supporting values
visit_length_p25 = ValueSpec(name="visit_length_p25", 
                             unit="Minutes", 
                             value_kind=ValueKind.NUMBER, 
                             display_name="25th Percentile of Visit Length (Min)", 
                             description="25th percentile of estimated time spent visiting the venue per person in minutes."
                            )

visit_length_p75 = ValueSpec(name="visit_length_p75", 
                             unit="Minutes", 
                             value_kind=ValueKind.NUMBER, 
                             display_name="75th Percentile of Visit Length (Min)", 
                             description="75th percentile of estimated time spent visiting the venue per person in minutes."
                            )

In [ ]:
venue_visit_length = rwg.build_metric("rwg_venue_visit_length_quarter_202103") \
                               .with_layer(venue_layer) \
                               .with_value(visit_length_p50) \
                               .with_supporting_value(visit_length_p25) \
                               .with_supporting_value(visit_length_p75) \
                               .with_cadence(Cadence.QUARTERLY) \
                               .with_display_name("Median Venue Visit Length") \
                               .with_description("The estimated average length of time spent visiting the venue per person.") \
                               .create() 

GRPCError: ignored

In [ ]:
%%bigquery

  DECLARE START_DATE DATE DEFAULT "2018-10-01";

  CREATE OR REPLACE TABLE `uc-unacat.inbox.rwg_venue_visit_length_quarter_202103` AS (
    SELECT
      venue_id as feature_id,
      observation_start_date as observation_start,
      observation_end_date as observation_end,
      COALESCE(visit_length_p50, 0) as value_visit_length_p50,
      COALESCE(visit_length_p25, 0) as value_visit_length_p25,
      COALESCE(visit_length_p75, 0) as value_visit_length_p75
    FROM `uc-catalog.venue_package_202103_4d_lag.venue_visit_length`
    WHERE observation_start_date >= START_DATE
      #AND venue_id NOT IN ("84ba1736-181c-4be4-86cb-a1a514b4371e", "f8f3387d-72fc-490f-ac67-10fe7a2a2b2f", "70a6a8ed-855a-40ee-81d5-cd0d149e9bc5", "6075d9cc-9bed-448c-8895-87b9535c503c")
  )

Query successful. Cost $0.01


""


In [ ]:
%%bigquery

  select min(observation_start) as min_start, max(observation_end) as max_end from  `uc-unacat.inbox.rwg_venue_visit_length_quarter_202103`

Query successful. Cost $0.00


,min_start,max_end
0,2018-10-01,2021-03-31


In [ ]:
import datetime
venue_visit_length = rwg.metric("rwg_venue_visit_length_quarter_202103")
index_job = venue_visit_length.index(
    big_query_table_id="rwg_venue_visit_length_quarter_202103",
    start_date=datetime.date(2018,10,1),
    end_date=datetime.date(2020,12,31),
    change_set=False
)

In [ ]:
rwg.metric("rwg_venue_visit_length_quarter_202103").report()

MetricReport(observation_period=Period(start=Date(year=2018, month=10, day=1), end=Date(year=2020, month=12, day=31)), total_size=2656627)

---

## Analysis / Tests